In [2]:
import torch
import numpy as np

In [3]:
from model.renderer.panopli_tensoRF_renderer import TensoRFRenderer
from model.radiance_field.tensoRF import TensorVMSplit

In [4]:
# Copyright (c) Meta Platforms, Inc. All Rights Reserved
import sys
import random
import omegaconf
import torch
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from tqdm import tqdm
import numpy as np
from dataset import PanopLiDataset, create_segmentation_data_panopli
from model.radiance_field.tensoRF import TensorVMSplit
from model.renderer.panopli_tensoRF_renderer import TensoRFRenderer
from trainer import visualize_panoptic_outputs
from util.camera import distance_to_depth
from util.misc import get_parameters_from_state_dict

from util.distinct_colors import DistinctColors
from dataset.preprocessing.preprocess_scannet import get_thing_semantics

In [25]:
cfg = omegaconf.OmegaConf.load(Path('/home/rozenberszki/project/panoptic-lifting/pretrained-examples/scannet_scene042302/checkpoints/epoch=9-step=678528.ckpt').parents[1] / "config.yaml")
cfg.resume ='/home/rozenberszki/project/panoptic-lifting/pretrained-examples/scannet_scene042302/checkpoints/epoch=9-step=678528.ckpt'
test_mode = False if len(sys.argv) == 2 else sys.argv[2] == "True"
cfg.image_dim = [256, 384]
if isinstance(cfg.image_dim, int):
    cfg.image_dim = [cfg.image_dim, cfg.image_dim]
config = cfg
trajectory_name = "trajectory_blender"
test_only = True

In [26]:
output_dir = (
    Path("runs")
    / f"{Path(config.dataset_root).stem}_{trajectory_name if not test_only else 'test'}_{Path(config.experiment)}"
)
print(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)
device = torch.device("cuda:0")
test_set = PanopLiDataset(
    Path(config.dataset_root),
    "test",
    (config.image_dim[0], config.image_dim[1]),
    config.max_depth,
    overfit=config.overfit,
    semantics_dir="m2f_semantics",
    instance_dir="m2f_instance",
    instance_to_semantic_key="m2f_instance_to_semantic",
    create_seg_data_func=create_segmentation_data_panopli,
    subsample_frames=config.subsample_frames,
)
H, W, alpha = config.image_dim[0], config.image_dim[1], 0.65
# whether to render the test set or a predefined trajectory through the scene
if test_only:
    trajectory_set = test_set
else:
    trajectory_set = test_set.get_trajectory_set(trajectory_name, True)
trajectory_loader = DataLoader(
    trajectory_set, shuffle=False, num_workers=0, batch_size=1
)
checkpoint = torch.load(config.resume, map_location="cpu")
state_dict = checkpoint["state_dict"]
total_classes = len(test_set.segmentation_data.bg_classes) + len(
    test_set.segmentation_data.fg_classes
)
output_mlp_semantics = (
    torch.nn.Identity()
    if config.semantic_weight_mode != "softmax"
    else torch.nn.Softmax(dim=-1)
)
model = TensorVMSplit(
    [config.min_grid_dim, config.min_grid_dim, config.min_grid_dim],
    num_semantics_comps=(32, 32, 32),
    num_semantic_classes=total_classes,
    dim_feature_instance=config.max_instances,
    output_mlp_semantics=output_mlp_semantics,
    use_semantic_mlp=config.use_mlp_for_semantics,
)
renderer = TensoRFRenderer(
    test_set.scene_bounds,
    [config.min_grid_dim, config.min_grid_dim, config.min_grid_dim],
    semantic_weight_mode=config.semantic_weight_mode,
)
renderer.load_state_dict(get_parameters_from_state_dict(state_dict, "renderer"))
for epoch in config.grid_upscale_epochs[::-1]:
    if checkpoint["epoch"] >= epoch:
        model.upsample_volume_grid(renderer.grid_dim)
        renderer.update_step_size(renderer.grid_dim)
        break
model.load_state_dict(get_parameters_from_state_dict(state_dict, "model"))
model = model.to(device)
renderer = renderer.to(device)
print('')

runs/scene0423_02_test_01170151_PanopLi_scannet042302_electrical-forest

[00] aabb tensor([-1., -1., -1.,  1.,  1.,  1.])
[00] grid size tensor([128, 128, 128])
[00] units:  None
[00] sampling step size:  tensor(0.0079)
[00] sampling number:  440

[00] aabb tensor([-1.0000, -0.9528, -0.0866,  1.0000,  1.0000,  0.3071])
[00] grid size tensor([332, 324,  65])
[00] units:  tensor(0.0079)
[00] sampling step size:  tensor(0.0030)
[00] sampling number:  929



In [48]:
with torch.no_grad():
        # create a new model for background
        chunk_size = int(2 ** 16)
        alpha, dense_xyz = renderer.get_dense_alpha(model)
        xyz_sampled = renderer.normalize_coordinates(dense_xyz)
        outputs_semantic = []
        outputs_instance = []
        output_colors = []
        for chunk in tqdm(torch.split(xyz_sampled.view(-1, 3), chunk_size), desc='grid_conv'):
            outputs_semantic.append(model.render_semantic_mlp(None, model.compute_semantic_feature(chunk)).cpu())
            outputs_instance.append(model.compute_instance_feature(chunk).cpu())
            dir_0 = torch.zeros_like(chunk)
            dir_a, dir_b, dir_c = dir_0.clone(), dir_0.clone(), dir_0.clone()
            dir_a[:, 0] = 1
            dir_b[:, 1] = 1
            dir_c[:, 2] = 1
            total = model.render_appearance_mlp(dir_a, model.compute_appearance_feature(chunk)).cpu() + model.render_appearance_mlp(dir_b, model.compute_appearance_feature(chunk)).cpu() + model.render_appearance_mlp(dir_c, model.compute_appearance_feature(chunk)).cpu()
            total = total / 3
            output_colors.append(total)
        sem_labels = torch.cat(outputs_semantic, dim=0).reshape([xyz_sampled.shape[0], xyz_sampled.shape[1], xyz_sampled.shape[2], -1]).argmax(dim=-1).int().transpose(0, 2).contiguous()
        colors = torch.cat(output_colors, dim=0).reshape([xyz_sampled.shape[0], xyz_sampled.shape[1], xyz_sampled.shape[2], -1]).float().transpose(0, 2).contiguous()
        dense_xyz = dense_xyz.transpose(0, 2).contiguous()
        alpha = alpha.clamp(0, 1).transpose(0, 2).contiguous()
        alpha_thres = 0.01 ## no idea set by me, Julius
        alpha[alpha >= alpha_thres] = 1
        alpha[alpha < alpha_thres] = 0
        mask = alpha > 0.5
        valid_xyz = dense_xyz[mask]
        distinct_colors = DistinctColors()
        semantic_bg = torch.from_numpy(np.isin(sem_labels.cpu().numpy(), [i for i, x in enumerate(get_thing_semantics("extended")) if not x])).to(sem_labels.device)
        thing_semantics = sem_labels.clone()
        thing_semantics[semantic_bg] = 0
        colored_semantics = distinct_colors.get_color_fast_numpy(sem_labels.cpu().numpy().reshape(-1)).reshape(list(sem_labels.shape) + [3])
        print('')


grid_conv: 100%|██████████| 107/107 [00:01<00:00, 63.21it/s]


In [49]:
valid_xyz.shape

torch.Size([1421244, 3])

In [50]:
alpha.shape, colored_semantics.shape

(torch.Size([65, 324, 332]), (65, 324, 332, 3))

In [51]:
normalized_mc(alpha.transpose(0, 2), torch.from_numpy(colored_semantics).transpose(0, 2), renderer.bbox_aabb).export('semantics.obj')

NameError: name 'normalized_mc' is not defined

In [ ]:
valid_xyz.shape, colored_semantics.shape

(torch.Size([598242, 3]), (65, 324, 332, 3))

In [52]:
import trimesh
import numpy as np
import torch

# Assuming your tensor is named 'points'
points = valid_xyz.detach().cpu() # Replace with your actual tensor
# Convert the tensor to a numpy array
points_np = points.numpy()

# Create a trimesh object from the numpy array
mesh = trimesh.Trimesh(vertices=points_np)

# Save the mesh as a .ply file
mesh.export('output_scannet.ply', file_type='ply')
print('')

In [9]:
gt = trimesh.load("/home/rozenberszki/scene0423_02/scene0423_02_vh_clean_2.ply")

In [10]:
gt.vertices.shape

(60071, 3)